<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
IC TABLES - GENERATION</p><br>

In [2]:
# General importings
import os
import json
import random

import invisible_cities.database.load_db as db

In [13]:
# Detectors: "Tst", "Next100", "Flex100", "FlexDens"

det_name = "GaP_ext"

In [14]:
tables = [
    "ChannelGain",
    "ChannelMapping",
    "ChannelMask",
    "ChannelPosition",
    "DetectorGeo",
    "PMTFELowFrequencyNoise",
    "PMTFEMapping",
    "PmtBlr",
    "PmtNoiseRms"
]

In [85]:
def load_detector_config(det_name, min_run, max_run):
    config_fname = f"config/{det_name.replace('_ext','')}.config"

    with open(config_fname) as config_file:
        det_config = json.load(config_file)
        det_config['MinRun'] = min_run
        det_config['MaxRun'] = max_run-1
    
    # Getting sensors info
    sensor_labels = det_config["SensorLabels"]
    for label in sensor_labels:
        det_config[label] = []
    
    sensors_fname = det_config["SensorFile"].replace('rmin', f'{min_run}').replace('rmax', f'{max_run}')
    with open(f"config/GaP_gains/{sensors_fname}", 'r') as sensors_file:
        for entry in sensors_file.read().splitlines():
            words = entry.split()
            #ext_pmt = words[0]
            if (len(words) and (words[0] in sensor_labels)):
                sensor_id = int(words[1])
                pos = words[2][1:-1].split(',')
                sensor_x = float(pos[0])
                sensor_y = float(pos[1])
                gain = words[3][1:-1].split(',')
                sensor_gain      = float(gain[0])
                sensor_gain_err  = float(gain[1])
                sensor_sigma     = float(gain[2])
                sensor_sigma_err = float(gain[3])
                #print(words[0])
                det_config[words[0]].append((sensor_id, sensor_x, sensor_y,
                                             sensor_gain , sensor_gain_err,
                                             sensor_sigma, sensor_sigma_err))
        
    if '_ext' in det_name:
        det_config['PMT'].append((7, 0, 0,
                                     1, 0,
                                     0, 0))
    #print(det_config)
    return det_config

In [80]:
def get_ChannelGain_entries(det_config):
    
    template = open("templates/ChannelGain.entry").read()
    
    
    MinRun = det_config["MinRun"]
    MaxRun = det_config["MaxRun"]
    
    entries = ""
    for label in det_config["SensorLabels"]:
        entries += "\n"
        for sensor in det_config[label]:
            SensorID      = sensor[0]
            Centroid      = sensor[3]
            ErrorCentroid = sensor[4]
            Sigma         = sensor[5]
            ErrorSigma    = sensor[6]
            params        = locals()
            entries      += template.format(**params) + "\n"

    return entries

In [81]:
def get_ChannelMapping_entries(det_config):
    
    template = open("templates/ChannelMapping.entry").read()
    
    MinRun = det_config["MinRun"]
    MaxRun = det_config["MaxRun"]

    entries = ""
    for label in det_config["SensorLabels"]:
        entries += "\n"
        for sensor in det_config[label]:
            SensorID = sensor[0]
            ElecID   = SensorID
            params   = locals()
            entries += template.format(**params) + "\n"

    return entries

In [82]:
def get_ChannelPosition_entries(det_config):
    
    template = open("templates/ChannelPosition.entry").read()
    
    MinRun = det_config["MinRun"]
    MaxRun = det_config["MaxRun"]

    entries = ""
    for label in det_config["SensorLabels"]:
        entries += "\n"
        for sensor in det_config[label]:
            SensorID      = sensor[0]
            Label         = label
            Type          = det_config[label + "_Type"]
            X             = sensor[1]
            Y             = sensor[2]
            params        = locals()
            entries      += template.format(**params) + "\n"
            #print(entries)
    return entries

In [83]:
def get_DetectorGeo_entries(det_config):
    
    template = open("templates/DetectorGeo.entry").read()
    
    XMIN = det_config["XMIN"]
    XMAX = det_config["XMAX"]
    YMIN = det_config["YMIN"]
    YMAX = det_config["YMAX"]
    ZMIN = det_config["ZMIN"]
    ZMAX = det_config["ZMAX"]
    RMAX = det_config["RMAX"]

    params  = locals()
    entries = template.format(**params) + "\n"

    return entries

In [84]:
### Reading Detector Configuration

runs = [0, 235, 620, 701, 1073, 10000]

### PATHs
tPath = "./templates"
oPath = f"{det_name}"
if not os.path.isdir(oPath):
    os.makedirs(oPath)

### Generating tables
for table in tables:
    
    print(f"Generating table {table} ...")
    table_fname  = f"{oPath}/{table}.sql"

    if table == "SipmNoisePDF":
        generate_SipmNoisePDF_DB(tPath, table_fname)
    else:
        # Getting header
        header_fname = f"{tPath}/{table}.h"
        with open(header_fname, 'r') as header_file:
            content = header_file.read() + "\n"
            
        for r0, r1 in zip(runs, runs[1:]):
            det_config = load_detector_config(det_name, r0, r1)
            
        # Add table registers
            if table   == "ChannelGain"    : content += get_ChannelGain_entries(det_config)
            elif table == "ChannelMapping" : content += get_ChannelMapping_entries(det_config)
            elif table == "ChannelPosition": content += get_ChannelPosition_entries(det_config)
            elif table == "DetectorGeo"    : content += get_DetectorGeo_entries(det_config)
            elif table == "SipmBaseline"   : content += get_SipmBaseline_entries(det_config)
    
            # Writing table
            with open(table_fname, 'w') as table_file:
                table_file.write(content)
            #print(f"Runs {r0} to {r1} finished")  
            
print("Done !!")

Generating table ChannelGain ...
{'MinRun': 0, 'MaxRun': 234, 'XMIN': -80, 'XMAX': 80, 'YMIN': -80, 'YMAX': 80, 'ZMIN': 0, 'ZMAX': 300, 'RMAX': 80, 'SensorFile': 'GaP.sensors_MinRun_rmin_MaxRun_rmax.txt', 'SensorLabels': ['PMT'], 'PMT_Type': 'Anode', 'PMT': [(0, 0.0, 0.0, 29.83, 0.39, 14.7, 0.42), (1, 36.253, 2.949, 15.27, 0.46, 12.81, 0.33), (2, -15.573, -32.871, 25.05, 0.53, 12.28, 0.57), (3, -20.68, 29.922, 22.21, 0.23, -10.68, 0.26), (4, 20.68, -29.922, 25.46, 0.9, 15.43, 0.96), (5, -36.253, -2.949, 34.77, 0.65, 18.18, 0.7), (6, 15.573, 32.871, 31.02, 0.56, 19.0, 0.6), (7, 0, 0, 1, 0, 0, 0)]}
{'MinRun': 235, 'MaxRun': 619, 'XMIN': -80, 'XMAX': 80, 'YMIN': -80, 'YMAX': 80, 'ZMIN': 0, 'ZMAX': 300, 'RMAX': 80, 'SensorFile': 'GaP.sensors_MinRun_rmin_MaxRun_rmax.txt', 'SensorLabels': ['PMT'], 'PMT_Type': 'Anode', 'PMT': [(0, 0.0, 0.0, 22.76, 0.34, 11.89, 0.38), (1, 36.253, 2.949, 12.17, 0.31, 9.35, 0.24), (2, -15.573, -32.871, 23.66, 0.35, 11.87, 0.39), (3, -20.68, 29.922, 17.01, 0.25, 

In [60]:
table_fname

'GaP_ext/PmtNoiseRms.sql'